<a href="https://colab.research.google.com/github/madeleinegates/Transportation-Modeling-RNN/blob/main/embedded_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd 
import keras
import numpy as np
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.losses import BinaryCrossentropy

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
import matplotlib
import numpy
from numpy import concatenate

from numpy import argmax
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from numpy import sqrt
from numpy import asarray
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
numpy.random.seed(1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from ipykernel import kernelapp as app


In [24]:
train = pd.read_csv('/content/drive/My Drive/training_laststep.csv')
train.head()

,trip_n,id,y_last,y_last_0,y_last_1,y_last_2,y_last_3,y_last_4,y,year,...,child_fare,free_fare,over_16_fare,no_car,under_one_car,at_least_one_car,snow,no_rain,light_rain,rain
0,0,1,4,0,0,0,0,1,3,2012,...,0,0,0,0,0,1,0,1,0,0
1,1,1,3,0,0,0,1,0,3,2012,...,0,0,0,0,0,1,0,1,0,0
2,2,1,3,0,0,0,1,0,3,2012,...,0,0,0,0,0,1,0,1,0,0
3,3,1,3,0,0,0,1,0,3,2012,...,0,0,0,0,0,1,0,1,0,0
4,2,2,4,0,0,0,0,1,3,2012,...,0,0,0,0,0,1,0,1,0,0


In [25]:
y_train = train['y'].tolist()
y_train = np.array(y_train)
y_train.shape

(70913,)

In [26]:
dev = pd.read_csv('/content/drive/My Drive/dev_laststep.csv')
dev.head()

,trip_n,id,y_last,y_last_0,y_last_1,y_last_2,y_last_3,y_last_4,y,year,...,child_fare,free_fare,over_16_fare,no_car,under_one_car,at_least_one_car,snow,no_rain,light_rain,rain
0,1,1,4,0,0,0,0,1,0,2012,...,0,1,0,1,0,0,0,1,0,0
1,0,2,4,0,0,0,0,1,2,2012,...,0,0,0,1,0,0,0,1,0,0
2,1,2,2,0,0,1,0,0,2,2012,...,0,0,0,1,0,0,0,1,0,0
3,0,3,4,0,0,0,0,1,2,2012,...,1,0,0,1,0,0,0,1,0,0
4,1,3,2,0,0,1,0,0,2,2012,...,1,0,0,1,0,0,0,1,0,0


In [27]:
test = pd.read_csv('/content/drive/My Drive/test_laststep.csv')
test.head()

,trip_n,id,y_last,y_last_0,y_last_1,y_last_2,y_last_3,y_last_4,y,year,...,child_fare,free_fare,over_16_fare,no_car,under_one_car,at_least_one_car,snow,no_rain,light_rain,rain
0,0,1,4,0,0,0,0,1,3,2012,...,0,1,0,0,0,1,0,1,0,0
1,1,1,3,0,0,0,1,0,3,2012,...,0,1,0,0,0,1,0,1,0,0
2,2,1,3,0,0,0,1,0,3,2012,...,0,1,0,0,0,1,0,1,0,0
3,3,1,3,0,0,0,1,0,3,2012,...,0,1,0,0,0,1,0,1,0,0
4,0,2,4,0,0,0,0,1,2,2012,...,0,0,1,0,0,1,0,1,0,0


In [28]:
y_dev = dev['y'].tolist()
y_dev = np.array(y_dev)
y_dev.shape

(5104,)

In [29]:
y_test = test['y'].tolist()
y_test = np.array(y_test)
y_test.shape

(5069,)

In [30]:
train_lasttrip = train.iloc[:, 3:8]
dev_lasttrip = dev.iloc[:, 3:8]
test_lasttrip = test.iloc[:, 3:8]

In [31]:
train_weather = train.iloc[:, 33:]
train_weather.drop(train_weather.iloc[:, 5:18], inplace=True, axis=1)
train_weather.drop(train_weather.iloc[:, :4], inplace=True, axis=1)

train_weather.head()

,drv_hr_sun_dur,snow,no_rain,light_rain,rain
0,1.0,0,1,0,0
1,1.0,0,1,0,0
2,1.0,0,1,0,0
3,1.0,0,1,0,0
4,1.0,0,1,0,0


In [32]:
dev_weather = dev.iloc[:, 33:]
dev_weather.drop(dev_weather.iloc[:, 5:18], inplace=True, axis=1)
dev_weather.drop(dev_weather.iloc[:, :4], inplace=True, axis=1)
dev_weather.head()

,drv_hr_sun_dur,snow,no_rain,light_rain,rain
0,1.0,0,1,0,0
1,0.6,0,1,0,0
2,0.0,0,1,0,0
3,0.6,0,1,0,0
4,0.0,0,1,0,0


In [33]:
test_weather = test.iloc[:, 33:]
test_weather.drop(test_weather.iloc[:, 5:18], inplace=True, axis=1)
test_weather.drop(test_weather.iloc[:, :4], inplace=True, axis=1)
test_weather.head()

,drv_hr_sun_dur,snow,no_rain,light_rain,rain
0,1.0,0,1,0,0
1,1.0,0,1,0,0
2,1.0,0,1,0,0
3,0.0,0,1,0,0
4,0.1,0,1,0,0


In [34]:
 x_train = pd.concat([train.iloc[:, 38:51], train.iloc[:, 10:33], train.iloc[:, 33:37]], axis = 1)
 x_dev = pd.concat([dev.iloc[:, 38:51], dev.iloc[:, 10:33], dev.iloc[:, 33:37]], axis = 1)
 x_test = pd.concat([test.iloc[:, 38:51], test.iloc[:, 10:33], test.iloc[:, 33:37]], axis = 1)

In [35]:
print(x_train.shape, train_weather.shape, train_lasttrip.shape, y_train.shape)
print(x_dev.shape, dev_weather.shape, dev_lasttrip.shape, y_dev.shape)
print(x_test.shape, test_weather.shape, test_lasttrip.shape, y_test.shape)

(70913, 40) (70913, 5) (70913, 5) (70913,)
(5104, 40) (5104, 5) (5104, 5) (5104,)
(5069, 40) (5069, 5) (5069, 5) (5069,)


In [38]:
input_weather = keras.layers.Input(shape=(train_weather.shape[1],))
input_lasttrip = keras.layers.Input(shape=(train_lasttrip.shape[1],))
input_x = keras.layers.Input(shape=(x_train.shape[1],))

# Weather embedding layer
emb_weather = keras.layers.Embedding(input_dim = train_weather.shape[1], output_dim=3)(input_weather) 
flatten_weather =  keras.layers.Flatten()(emb_weather)
# Last trip embedding layer
emb_lasttrip = keras.layers.Embedding(input_dim = train_lasttrip.shape[1], output_dim=3)(input_lasttrip) 
flatten_lasttrip =  keras.layers.Flatten()(emb_lasttrip)
# Concatenate the three layers
# 
conc = keras.layers.Concatenate()([flatten_weather, flatten_lasttrip, input_x])
# Add additional dense layers
dense1 = keras.layers.Dense(16, activation=tf.nn.relu,)(conc) #13?
dense2 = keras.layers.Dense(12, activation=tf.nn.relu,)(dense1)
dense3 = keras.layers.Dense(10, activation=tf.nn.relu,)(dense2)
dense4 = keras.layers.Dense(8, activation=tf.nn.relu,)(dense3)

# Creating output layer
out = keras.layers.Dense(4, activation=tf.nn.softmax)(dense4)
# Creating model
model = keras.Model(inputs=[input_weather, input_lasttrip, input_x], outputs= out)

#model.compile(loss='binary_crossentropy', optimizer='adam')

In [39]:
model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 5, 3)         15          ['input_7[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 5, 3)         15          ['input_8[0][0]']                
                                                                                            

In [40]:
model.fit([train_weather, train_lasttrip, x_train], y_train, epochs = 100, batch_size = 32)

Epoch 1/100
2217/2217 [==============================] - 7s 2ms/step - loss: 0.6493 - accuracy: 0.7509
Epoch 2/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5400 - accuracy: 0.8002
Epoch 3/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5269 - accuracy: 0.8059
Epoch 4/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5221 - accuracy: 0.8075
Epoch 5/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5172 - accuracy: 0.8087
Epoch 6/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5128 - accuracy: 0.8096
Epoch 7/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5103 - accuracy: 0.8106
Epoch 8/100
2217/2217 [==============================] - 5s 2ms/step - loss: 0.5085 - accuracy: 0.8114
Epoch 9/100
2217/2217 [==============================] - 4s 2ms/step - loss: 0.5071 - accuracy: 0.8120
Epoch 10/100
2217/2217 [==============================] - 4s 2ms/step - l

In [41]:
loss, accuracy = model.evaluate([dev_weather, dev_lasttrip, x_dev], y_dev, verbose=0)
print(loss, accuracy)

0.5695577263832092 0.8089733719825745


In [42]:
loss, accuracy = model.evaluate([test_weather, test_lasttrip, x_test], y_test, verbose=0)
print(loss, accuracy)

0.5607520937919617 0.8102189898490906
